In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import ultraplot as uplt
from scipy import stats
from Py6S import *

In [ ]:
SixS.test()

# Hotspot Identification

In [ ]:
pal_rad_path = 'datasets\palisades_fire\AV320250111t210400_005_L1B_RDN_3f4aef90_RDN.nc'
pal_mask_path = 'datasets\palisades_fire\AV320250111t210400_005_L1B_RDN_3f4aef90_BANDMASK.nc'
pal_ds = xr.open_datatree(pal_rad_path)
pal_ds

In [ ]:
pal_obs_path = 'datasets\palisades_fire\AV320250111t210400_005_L1B_ORT_8827a51f_OBS.nc'
obs_ds = xr.open_datatree(pal_obs_path) # Observational parameters
obs_ds 

In [ ]:
samples_coords = np.arange(1234)
lines_coords = np.arange(1280)
# Assign dummy coordinates
pal_radiance = pal_ds.radiance.radiance.assign_coords({'samples':samples_coords, 'lines':lines_coords})
pal_radiance

In [ ]:
# observation parameters
obs_params = obs_ds.observation_parameters.to_dataset().assign_coords({'samples':samples_coords, 'lines':lines_coords})
obs_params

# Sample Images/Plots

In [ ]:
# Generate an RGB image
def normalize(band):
    band_min = band.min()
    band_max = band.max()
    return (band - band_min) / (band_max - band_min)

red_ = pal_radiance.sel(wavelength=700, method='nearest')
green_ = pal_radiance.sel(wavelength=550, method='nearest')
blue_ = pal_radiance.sel(wavelength=400, method='nearest')

red = normalize(red_)
green = normalize(green_) 
blue = normalize(blue_)
rgb_image = np.dstack((red.values, green.values, blue.values))
plt.figure(figsize=(5, 5))
plt.imshow(rgb_image)

In [ ]:
red_ = pal_radiance.sel(wavelength=2200, method='nearest')
green_ = pal_radiance.sel(wavelength=700, method='nearest')
blue_ = pal_radiance.sel(wavelength=550, method='nearest')

red = normalize(red_)
green = normalize(green_)
blue = normalize(blue_)
rgb_image = np.dstack((red.values, green.values, blue.values))
plt.figure(figsize=(5, 5))
plt.imshow(rgb_image)

In [ ]:
# Visualize the observation parameters
fig, ax = uplt.subplots(ncols=2)
obs_params.path_length.plot(ax=ax[0])
ax[0].format(
    title='Path Length',
    yreverse=True,
)

obs_params.cosine_i.plot(ax=ax[1])
ax[1].format(
    title='cosine_i',
    yreverse=True,
)

In [ ]:
# Visualize the observation parameters
fig, axs = uplt.subplots(nrows=2, ncols=2)

ax = axs[0,0]
obs_params.to_sun_zenith.plot(ax=ax)
ax.format(
    title='Solar Zenith Angle',
    yreverse=True,
)

ax = axs[0,1]
obs_params.to_sun_azimuth.plot(ax=ax)
ax.format(
    title='Solar Azimuth Angle',
    yreverse=True,
)

ax = axs[1,0]
obs_params.to_sensor_zenith.plot(ax=ax)
ax.format(
    title='Sensor Zenith Angle',
    yreverse=True,
)

ax = axs[1,1]
obs_params.to_sensor_azimuth.plot(ax=ax)
ax.format(
    title='Sensor Azimuth Angle',
    yreverse=True,
)

Both the solar zenith angle and azimuth angle can be treated as approximately constant throughout the read area.
As for the sensor zenith angle and azimuth angle, there is considerable variation in the angles across the sample dimension, because the sensor is read from a pushbroom sensor on an aircraft. Therefore from an aircraft altitude, the line-of-sight angle from left-to-right across the pushbroom varies significantly.

# Simplified Atmospheric Correction

In [ ]:
# From path length we assume an altitude of 5km

# HFDI Hotspot

In [ ]:
# Calculate the HDFI index
pal_rad_2430 = pal_radiance.sel(wavelength=slice(2420,2440)).mean(dim='wavelength')
pal_rad_2060 = pal_radiance.sel(wavelength=slice(2050,2070)).mean(dim='wavelength')
pal_HFDI = (pal_rad_2430 - pal_rad_2060)/(pal_rad_2430 + pal_rad_2060)

In [ ]:
fig, ax = uplt.subplots(refwidth=6)
pal_HFDI.plot(ax=ax, vmin=-.5, vmax=.5, discrete=False, cmap='RdBu_r')
ax.format(
    yreverse=True,
    suptitle='Palisades Fire 2025-01-11 HFDI Index'
)

In [ ]:
bins = np.linspace(-0.4, 0.4, 500)
fig, ax = uplt.subplots(refwidth=6, refaspect=(3,1))
_ = pal_HFDI.plot.hist(bins=bins, ax=ax)
ax.format(
    suptitle='Distribution of pixel HFDI'
)

This distribution looks like a skew-normal with a long-tail anomaly. Assume that background pixels follow a skew-normal distribution, and use this to determine an appropriate threshold.

In [ ]:
# Truncate the long tail
pal_dist = pal_HFDI.values.flatten()
pal_HFDI_trunc = pal_dist[pal_dist < -0.06] 

In [ ]:
# Retrieve the histogram counts
hist, bin_edges = np.histogram(pal_HFDI_trunc, bins=bins, density=True)
bin_centers = .5 * (bin_edges[1:] + bin_edges[:-1])
# Stack all values into one single vector
hist_original, bin_edges_original = np.histogram(pal_dist, bins=bins, density=True)
bin_centers_original = .5 * (bin_edges_original[1:] + bin_edges_original[:-1])

In [ ]:
result = stats.skewnorm.fit(
    data=pal_HFDI_trunc,
)

In [ ]:
shape, loc, scale = result
print(f"Shape (a): {shape}, Location (loc): {loc}, Scale (scale): {scale}")

In [ ]:
# Plot the fitted distribution
skewnorm_pdf = stats.skewnorm.pdf(x=bin_centers, a=shape, loc=loc, scale=scale)

In [ ]:
fig, ax = uplt.subplots()
ax.plot(bin_centers_original, hist_original)
ax.plot(bin_centers, 0.92 * skewnorm_pdf)

In [ ]:
# Define a fire mask
fire_mask = (pal_HFDI>-0.03)

In [ ]:
fig, ax = uplt.subplots(refwidth=6)
fire_mask.plot(ax=ax, vmin=-.5, vmax=.5, discrete=False, cmap='RdBu_r')
ax.format(
    yreverse=True,
    suptitle='Palisades Fire 2025-01-11 HFDI > -0.03'
)

# Test with individual pixels

In [ ]:
# choose a fire pixel
fire_pixel = pal_radiance.sel(samples=1200, lines=50)
print('Fire pixel HFDI:', pal_HFDI.sel(samples=1200, lines=50).values)
# choose an unburnt pixel
unburnt_pixel = pal_radiance.sel(samples=200, lines=600)
print('Unburnt pixel HFDI:', pal_HFDI.sel(samples=200, lines=600).values)

In [ ]:
# Plot the test spectra
fig, ax = uplt.subplots(refwidth=5, refaspect=(2,1))
ax.plot(fire_pixel, label='Fire pixel')
ax.plot(unburnt_pixel, label='Unburnt pixel')
ax.legend()